In [21]:
from dotenv import load_dotenv
from openai import OpenAI
import openai
import os
import sys
from pypdf import PdfReader
import gradio as gr

In [22]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Ensure the API keys are loaded correctly
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set. Please set it in your .env file.")
else:
    openai.api_key = openai_api_key

if gemini_api_key is None:
    raise ValueError("GEMINI_API_KEY environment variable not set. Please set it in your .env file.")
else:
    # Return both API keys
    (openai_api_key, gemini_api_key)


In [4]:
opneai = OpenAI()

In [6]:
reader = PdfReader("Profile.pdf")
def read_pdf(file):
    text = ""
    with open(file, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            text += page.extract_text()
    return text


In [18]:
from IPython.display import Markdown, display

print("Reading PDF...")
linkedin = read_pdf("Profile.pdf")
display(Markdown(linkedin))

Reading PDF...


   
Contact
Paris - France
+33766654028 (Mobile)
paulinocristovao@yahoo.com.
br
www.linkedin.com/in/paulino-
cristovao-aa685986 (LinkedIn)
sciam.fr/en/ (Personal)
Top Skills
Network Engineering
Python (Programming Language)
Microsoft Azure Machine Learning
Languages
Portuguese (Native or Bilingual)
Spanish (Professional Working)
English (Native or Bilingual)
French (Professional Working)
Japanese (Limited Working)
Paulino Cristovao
Data Scientist | Machine Learning | AI Engineer | Data Engineer
Courbevoie, Île-de-France, France
Experience
SCIAM
Data Engineer
June 2023 - Present (2 years)
Paris, Île-de-France, France
Technologies
We identify, design, and implement innovative and robust technological
solutions to solve our clients’ concrete problems.
Behavioral science
We analyze cognitive behaviors to understand the inner workings of individuals
and organizations, and to optimize their decision-making processes, habits,
and use of technology.
Organizations
In partnership with the world's first school of collective intelligence, we are
inventing and testing new methods to effectively solve today's and tomorrow’s
complex problems.
University of Poitiers
Postdoctoral Researcher
April 2022 - June 2023 (1 year 3 months)
France
iSheriff
Tech Support
May 2014 - June 2023 (9 years 2 months)
Technical Support
Total Defense Inc.
IT
August 2013 - June 2023 (9 years 11 months)
customers technical support
Totaldefense
  Page 1 of 2   
TDI
August 2012 - June 2023 (10 years 11 months)
Technical support
iYogi Inc.
iyogi Technical service
August 2012 - June 2023 (10 years 11 months)
Technical Support
Education
University of Tsukuba
Doctor of Philosophy - PhD, Computer Science · (2019 - 2022)
University of Poitiers
Posdoctoral, Machine Learniing · (April 2022 - April 2023)
University of Tsukuba
Master of Computer Science, Machine Learning · (2016 - 2019)
University of Pune
Bachelor of Science (BSc), Computer Science · (2009 - 2013)
Mercury Solutions
Diploma, IT security
  Page 2 of 2

In [13]:
with open("Profile.txt", "r", encoding="utf-8") as f:
    summary = f.read() 

In [14]:
display(Markdown(summary))

   
Contact
Paris - France
+33766654028 (Mobile)
paulinocristovao@yahoo.com.
br
www.linkedin.com/in/paulino-
cristovao-aa685986 (LinkedIn)
sciam.fr/en/ (Personal)
Top Skills
Network Engineering
Python (Programming Language)
Microsoft Azure Machine Learning
Languages
Portuguese (Native or Bilingual)
Spanish (Professional Working)
English (Native or Bilingual)
French (Professional Working)
Japanese (Limited Working)
Paulino Cristovao
Data Scientist | Machine Learning | AI Engineer | Data Engineer
Courbevoie, Île-de-France, France
Experience
SCIAM
Data Engineer
June 2023 - Present (2 years)
Paris, Île-de-France, France
Technologies
We identify, design, and implement innovative and robust technological
solutions to solve our clients’ concrete problems.
Behavioral science
We analyze cognitive behaviors to understand the inner workings of individuals
and organizations, and to optimize their decision-making processes, habits,
and use of technology.
Organizations
In partnership with the world's first school of collective intelligence, we are
inventing and testing new methods to effectively solve today's and tomorrow’s
complex problems.
University of Poitiers
Postdoctoral Researcher
April 2022 - June 2023 (1 year 3 months)
France
iSheriff
Tech Support
May 2014 - June 2023 (9 years 2 months)
Technical Support
Total Defense Inc.
IT
August 2013 - June 2023 (9 years 11 months)
customers technical support
Totaldefense
  Page 1 of 2   
TDI
August 2012 - June 2023 (10 years 11 months)
Technical support
iYogi Inc.
iyogi Technical service
August 2012 - June 2023 (10 years 11 months)
Technical Support
Education
University of Tsukuba
Doctor of Philosophy - PhD, Computer Science · (2019 - 2022)
University of Poitiers
Posdoctoral, Machine Learniing · (April 2022 - April 2023)
University of Tsukuba
Master of Computer Science, Machine Learning · (2016 - 2019)
University of Pune
Bachelor of Science (BSc), Computer Science · (2009 - 2013)
Mercury Solutions
Diploma, IT security
  Page 2 of 2

In [16]:
name = "Paulino"

In [19]:
system_prompt = f"""
You are a helpful assistant. You will be given a text about a person named {name}, and you will answer questions based on the provided information. 
If you don't know the answer, respond with "I don't know."

### Text:
{linkedin}

### Summary:
{summary}

Using this information, engage with the user and answer their questions accurately and concisely.
"""

In [20]:
print(system_prompt)


You are a helpful assistant. You will be given a text about a person named Paulino, and you will answer questions based on the provided information. 
If you don't know the answer, respond with "I don't know."

### Text:
   
Contact
Paris - France
+33766654028 (Mobile)
paulinocristovao@yahoo.com.
br
www.linkedin.com/in/paulino-
cristovao-aa685986 (LinkedIn)
sciam.fr/en/ (Personal)
Top Skills
Network Engineering
Python (Programming Language)
Microsoft Azure Machine Learning
Languages
Portuguese (Native or Bilingual)
Spanish (Professional Working)
English (Native or Bilingual)
French (Professional Working)
Japanese (Limited Working)
Paulino Cristovao
Data Scientist | Machine Learning | AI Engineer | Data Engineer
Courbevoie, Île-de-France, France
Experience
SCIAM
Data Engineer
June 2023 - Present (2 years)
Paris, Île-de-France, France
Technologies
We identify, design, and implement innovative and robust technological
solutions to solve our clients’ concrete problems.
Behavioral science
W

In [38]:
def get_chat_response_with_chatgpt(user_message, chat_history):
    """
    Send the full conversation (system prompt + prior messages + new user message)
    to OpenAI’s chat endpoint and return the assistant’s reply.
    
    :param user_message: the latest message from the user
    :param chat_history:  list of dicts, e.g.
                          [{"role": "user", "content": "..."},
                           {"role": "assistant", "content": "..."},
                           …]
    :returns: (reply_text, updated_history)
    """
    # ensure history is a list
    chat_history = chat_history or []

    # build the message list
    messages = [{"role": "system", "content": system_prompt}] + chat_history
    messages.append({"role": "user", "content": user_message})

    # call OpenAI
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=messages,
        temperature=0.7,
        max_tokens=512,
    )
    assistant_reply = response.choices[0].message.content

    # append the new turn to history and return
    chat_history.append({"role": "user",      "content": user_message})
    chat_history.append({"role": "assistant", "content": assistant_reply})
    return assistant_reply

In [42]:
def get_gemini_response(user_message, chat_history):
    """
    Send the full conversation (system prompt + prior messages + new user message)
    to Gemini’s chat API and return the assistant’s reply.
    
    :param user_message: the latest message from the user
    :param chat_history:  list of dicts, e.g.
                          [{"role": "user", "content": "..."},
                           {"role": "assistant", "content": "..."},
                           …]
    :returns: (reply_text, updated_history)
    """
    chat_history = chat_history or []

    messages = [{"role": "system", "content": system_prompt}] + chat_history
    messages.append({"role": "user", "content": user_message})

    client = OpenAI(
        api_key=gemini_api_key,
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    response = client.chat.completions.create(
        model="gemini-2.0-flash",
        messages=messages,
    )
    assistant_reply = response.choices[0].message.content

    # update history with both sides of the exchange
    chat_history.append({"role": "user",      "content": user_message})
    chat_history.append({"role": "assistant", "content": assistant_reply})
    return assistant_reply

In [39]:
gr.ChatInterface(get_chat_response_with_chatgpt, type="messages", title="Chat with Gemini", description="Ask questions about the LinkedIn profile.").launch()

* Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.


In [43]:
gr.ChatInterface(get_gemini_response, type="messages", title="Chat with Gemini", description="Ask questions about the LinkedIn profile.").launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
